In [ ]:
import json
import pandas as pd
# from tqdm.auto import tqdm
from pyspark.sql import SparkSession, functions as f, DataFrame as DF
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, FloatType
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate() 

In [ ]:
df = spark.read.csv("./data/used_cars_data_combine.csv", header=True).drop("_c0", "Unnamed: 0")
df.createOrReplaceTempView("df")

In [ ]:
null_cnt = df.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in df.columns]).toPandas()
threshold = null_cnt > 0.4 * df.count()
df = df.drop(*threshold.columns[threshold.iloc[0]])
with open("./data/useless.json", 'r') as useless:
    df = df.drop(*json.load(useless))
df.createOrReplaceTempView("df")

In [ ]:
def performance(engine: str):
    if engine is None:
        return [None, None]
    p = [int(i) for i in engine.replace(",","").split() if i.isdigit()]
    return p if len(p) == 2 else [None, None]

def hp_rpm(df: DF, col: str) -> DF:
    original = df.columns
    schema = StructType([StructField(f"{col}_hp", IntegerType(), True),
                         StructField(f"{col}_rpm", IntegerType(), True)])
    trans = f.udf(performance, schema)
    return df.withColumn("result", trans(df[col])).select(*(original + ["result.*"]))

for col in ["power", "torque"]:
    df = hp_rpm(df, col).drop(col)

df_drop = df.drop("power_hp")

In [ ]:
df_drop.printSchema()

In [ ]:
transmission = f.udf(lambda tr: "6" if tr in ["Automatic", "Continuously Variable Transmission", "Manual"] else tr, StringType())
df_extract = df_drop.withColumn("transmission_display", transmission(df["transmission_display"]))

In [ ]:
df_extract = df_extract.withColumn("is_new", df["is_new"].cast(BooleanType()).cast(IntegerType()))

In [ ]:
def extract_val(df: DF, cols: list) -> DF:
    for col in cols:
        df = df.withColumn(col, f.regexp_extract(col, r"(\d+(?:\.\d+)?)", 1).cast(FloatType()))
    return df

df_extract = extract_val(df_extract, ["fuel_tank_volume", "maximum_seating", "transmission_display"])

In [ ]:
df_extract = df_extract.dropna(subset="mileage")

In [ ]:
get_mean = lambda df, col: df.select(f.mean(col)).collect()[0].__getattr__(f"avg({col})")
hp_mean = get_mean(df_extract, "horsepower")
engine_displacement_mean = get_mean(df_extract, "engine_displacement")

In [ ]:
electric = {"engine_type": "Electric_Motor", 
            "transmission_display": float(6), 
            "horsepower": hp_mean, 
            "engine_displacement": engine_displacement_mean}

df_fill = df_extract
for key, item in electric.items():
   df_fill = df_fill.withColumn(key, f.when(f.col("fuel_type") == "Electric", item).otherwise(f.col(key)))

In [205]:
fill_null_cols = ["maximum_seating", 
                  "city_fuel_economy", 
                  "highway_fuel_economy", 
                  "horsepower", 
                  "engine_displacement", 
                  "fuel_tank_volume", 
                  "power_rpm", 
                  "torque_hp", 
                  "torque_rpm"]
fill_val = df_fill.select([f.mean(c).alias(c) for c in fill_null_cols]).toPandas()

In [211]:
test = df_fill
for key, item in fill_val.items():
    test = test.na.fill(value=item[0], subset=key)

In [218]:
test.count()

2855653

In [216]:
test.createOrReplaceTempView("table")
spark.sql(f'''
   select {', '.join([f"count({col})" for col in fill_null_cols])}
   from table
''').show()

+----------------------+------------------------+---------------------------+-----------------+--------------------------+-----------------------+----------------+----------------+-----------------+
|count(maximum_seating)|count(city_fuel_economy)|count(highway_fuel_economy)|count(horsepower)|count(engine_displacement)|count(fuel_tank_volume)|count(power_rpm)|count(torque_hp)|count(torque_rpm)|
+----------------------+------------------------+---------------------------+-----------------+--------------------------+-----------------------+----------------+----------------+-----------------+
|               2855653|                 2395048|                    2395048|          2711444|                   2711444|                2855653|         2855653|         2855653|          2855653|
+----------------------+------------------------+---------------------------+-----------------+--------------------------+-----------------------+----------------+----------------+-----------------+



In [213]:
test.printSchema()

root
 |-- body_type: string (nullable = true)
 |-- city_fuel_economy: string (nullable = true)
 |-- engine_displacement: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- fuel_tank_volume: float (nullable = false)
 |-- fuel_type: string (nullable = true)
 |-- highway_fuel_economy: string (nullable = true)
 |-- horsepower: string (nullable = true)
 |-- is_new: integer (nullable = true)
 |-- listing_color: string (nullable = true)
 |-- make_name: string (nullable = true)
 |-- maximum_seating: float (nullable = false)
 |-- mileage: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- price: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- transmission_display: double (nullable = true)
 |-- wheel_system: string (nullable = true)
 |-- year: string (nullable = true)
 |-- sold_date: string (nullable = true)
 |-- sold_date_CPI: string (nullable = true)
 |-- sold_date_Manheim: string (nullable = true)
 |-- power_rpm: integer (nu

In [ ]:
df_fill.createOrReplaceTempView("df")
spark.sql('''
   select count(engine_displacement)
   from df
''').show(100)